# Print pokemons

api: https://github.com/rubenvereecken/pokemongo-api

In [ ]:
import print_pokemons as pp
from pogo.pokedex import pokedex

In [ ]:
username, password, auth_type = pp.get_credentials_from_keyfile('keyfile')

In [ ]:
trainer = pp.get_trainer(username, password, auth_type)

In [ ]:
inventory = trainer.session.getInventory()
bag = pp.get_pokemons(trainer)

In [ ]:
pp.print_pokemons_by_iv(bag)

In [ ]:
pp.print_pokemons_by_number(bag)

## Count pokemons and possible evolves

In [ ]:
from collections import Counter

def print_most_common(bag, limit=None):
    counter = Counter([(x['name'], x['id']) for x in bag if pokedex.evolves.get(x['id'], 0) > 0])

    print('{:>15} | {:>5} | {:>7} | {:>7} | {:>8} | {}'.format('Name', 'Need', 'Candies', 'NumHave', 'Can Evo.', 'Curr. Evo.'))
    print('-'*70)

    total_possible = 0
    total_current = 0
    if limit is None:
        most_common = sorted(counter.most_common())
    else:
        most_common = sorted(counter.most_common(limit))

    for item, count in most_common:
        family = pokedex.families[item[1]]

        n_candies = inventory.candies.get(item[1], 0)
        need = pokedex.evolves.get(item[1], 0)

        possible = int(n_candies / need)
        curr = min(possible, count)
        print('{:>15} | {:>5} | {:>7} | {:>7} | {:>8} | {}'.format(item[0], need, n_candies, count, possible, curr))

        total_possible += possible
        total_current += curr

    print('\nTotal possible evolutions: {}'.format(total_possible))
    print('Current total evolutions: {}'.format(total_current))

In [ ]:
print_most_common(bag)